# 实例：GW Catalog 数据分析案例

## DataFrame的组合与聚合以及透视表生成


Pandas 中常用的组合和聚合方法（透视表）

- groupby() / apply() / agg()
- stack() / unstack()
- melted()
- pivot_table()

透视表（pivot table）是用来汇总其它表的数据。 首先把源表分组（grouping），然后对各组内数据做汇总操作如排序、平均、累加、计数或字符串连接等。 透视表用于数据处理，在数据可视化程序如电子表格或商业智能软件中常见。 

HINT: 透视表的分析是针对类别型字段而言的

下面我们通过一个综合一点的数据分析案例，巩固上面学过的操作，同时练习一下dataframe的组合、聚合以及透视表生成的操作。

- 数据案例：
    - The first Open Gravitational-wave Catalog (1-OGC) [GitHub](https://github.com/gwastro/1-ogc)
    - The forth Open Gravitational-wave Catalog (4-OGC) [GitHub](https://github.com/gwastro/4-ogc)

- 数据详情：
    - 参考上面👆🏻的链接

More related GW open data: https://iphysresearch.github.io/blog/project/gwda/#-awesome-data-release

## Load libs

In [1]:
# 标准姿势
import numpy as np
import pandas as pd

# REF: https://www.dataquest.io/blog/settingwithcopywarning/
pd.set_option('mode.chained_assignment', None)

# !pip install h5py
import h5py

from datetime import datetime

## Load data

In [2]:
!ls -lh *OGC*

-rw-r--r-- 1 root root 11M Dec  2 10:41 1-OGC.hdf
-rw-r--r-- 1 root root 11K Dec  2 07:53 4OGC_top.txt


In [3]:
!head ./4OGC_top.txt

# Number, Name, GpsTime, Obs, Triggered, Pastro, IFAR, SNR_H, SNR_L, SNR_V 
1  GW150914_095045  1126259462.4257812  HL  HL  1.0  16376.568630773567  19.934932708740234  13.047991752624512  -1.0
2  GW151012_095443  1128678900.4450684  HL  HL  1.0  897.2759041259671  6.88588285446167  6.577003479003906  -1.0
3  GW151226_033853  1135136350.649414  HL  HL  1.0  14799.661708193906  10.518603324890137  7.405562400817871  -1.0
4  GW170104_101158  1167559936.6000977  HL  HL  1.0  18235.991189523284  8.863738059997559  9.633432388305664  -1.0
5  GW170121_212536  1169069154.5761719  HL  HL  1.0  16.10953285293576  5.239262580871582  8.938467025756836  -1.0
6  GW170202_135657  1170079035.732666  HL  HL  0.86  0.5021361028513931  5.370537281036377  6.213018417358398  -1.0
7  GW170304_163753  1172680691.3659668  HL  HL  0.74  0.2489521012197277  4.637211799621582  7.021427631378174  -1.0
8  GW170403_230611  1175295989.2316895  HL  HL  0.72  0.2464015211198255  5.180370807647705  5.534319877624512  

In [5]:
with open('./4OGC_top.txt', 'r') as f:
    names = f.readline()
names

'# Number, Name, GpsTime, Obs, Triggered, Pastro, IFAR, SNR_H, SNR_L, SNR_V \n'

In [6]:
names[2:-2]

'Number, Name, GpsTime, Obs, Triggered, Pastro, IFAR, SNR_H, SNR_L, SNR_V'

In [8]:
names[2:-2].split(', ')

['Number',
 'Name',
 'GpsTime',
 'Obs',
 'Triggered',
 'Pastro',
 'IFAR',
 'SNR_H',
 'SNR_L',
 'SNR_V']

In [12]:
# txt => DataFrame
df_4OGC = pd.read_csv('./4OGC_top.txt', sep='  ', index_col='Number', names=names[2:-2].split(', '), skiprows=1, engine='python')
df_4OGC

,Name,GpsTime,Obs,Triggered,Pastro,IFAR,SNR_H,SNR_L,SNR_V
Number,,,,,,,,,
1,GW150914_095045,1.126259e+09,HL,HL,1.00,16376.568631,19.934933,13.047992,-1.000000
2,GW151012_095443,1.128679e+09,HL,HL,1.00,897.275904,6.885883,6.577003,-1.000000
3,GW151226_033853,1.135136e+09,HL,HL,1.00,14799.661708,10.518603,7.405562,-1.000000
4,GW170104_101158,1.167560e+09,HL,HL,1.00,18235.991190,8.863738,9.633432,-1.000000
5,GW170121_212536,1.169069e+09,HL,HL,1.00,16.109533,5.239263,8.938467,-1.000000
...,...,...,...,...,...,...,...,...,...
90,GW200305_084739,1.267433e+09,HLV,HL,0.59,0.019142,4.504479,6.129437,-1.000000
91,GW200306_093714,1.267523e+09,HL,HL,0.51,0.017822,5.450192,5.895027,-1.000000
92,GW200311_115853,1.267963e+09,HLV,HLV,1.00,816.945669,12.031339,9.871196,6.675977


In [13]:
# hdf5 => DataFrame
with h5py.File('./1-OGC.hdf', 'r') as catalog:
    df_1OGC = pd.DataFrame(catalog['bbh'][:])

## View data

- 查看导入后数据的头部数据，默认输出前5行。

In [14]:
df_1OGC.head()

,chunk,H1_snr,L1_reduced_chisq,L1_end_time,spin1z,H1_chisq_dof,L1_sg_chisq,L1_chisq_dof,L1_snr,H1_coa_phase,...,jd,mass1,name,mass2,H1_sg_chisq,spin2z,H1_sigmasq,tdr,pastro,odds
0,1.0,4.833010,-0.000000,1.126074e+09,0.735548,214,1.0,-100,4.890781,-2.768472,...,2.457278e+06,5.986125,b'150912+06:13:00UTC',5.216917,1.0,0.968694,18449686.0,0.000028,2.420347e-11,2.420347e-11
1,1.0,5.042901,-0.000000,1.126074e+09,-0.922598,186,1.0,-100,4.935375,2.565755,...,2.457278e+06,5.961657,b'150912+06:19:09UTC',4.068976,1.0,0.605111,12820608.0,0.000035,2.322201e-10,2.322201e-10
2,1.0,5.678490,0.000000,1.126074e+09,0.359088,48,1.0,48,4.572548,-0.776491,...,2.457278e+06,60.465069,b'150912+06:26:35UTC',2.944411,1.0,0.238403,24585170.0,0.000036,4.259693e-10,4.259693e-10
3,1.0,4.615423,-0.000000,1.126075e+09,0.838187,-100,1.0,-100,4.969444,1.009600,...,2.457278e+06,11.926704,b'150912+06:42:42UTC',7.567072,1.0,0.077454,41936048.0,0.000028,2.420347e-11,2.420347e-11
4,1.0,5.205948,0.973344,1.126076e+09,-0.132135,-100,1.0,212,5.520049,2.674877,...,2.457278e+06,5.438966,b'150912+06:46:35UTC',3.128970,1.0,-0.303846,9825482.0,0.000653,1.452786e-05,1.452807e-05


In [15]:
df_4OGC.head()

,Name,GpsTime,Obs,Triggered,Pastro,IFAR,SNR_H,SNR_L,SNR_V
Number,,,,,,,,,
1,GW150914_095045,1.126259e+09,HL,HL,1.0,16376.568631,19.934933,13.047992,-1.0
2,GW151012_095443,1.128679e+09,HL,HL,1.0,897.275904,6.885883,6.577003,-1.0
3,GW151226_033853,1.135136e+09,HL,HL,1.0,14799.661708,10.518603,7.405562,-1.0
4,GW170104_101158,1.167560e+09,HL,HL,1.0,18235.991190,8.863738,9.633432,-1.0
5,GW170121_212536,1.169069e+09,HL,HL,1.0,16.109533,5.239263,8.938467,-1.0


- 确认一下数据的规模

In [16]:
df_1OGC.shape, df_4OGC.shape

((12741, 27), (94, 9))

- 检查一下导入的数据情况

In [17]:
df_1OGC.info(), df_4OGC.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12741 entries, 0 to 12740
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   chunk             12741 non-null  float64
 1   H1_snr            12741 non-null  float32
 2   L1_reduced_chisq  12741 non-null  float64
 3   L1_end_time       12741 non-null  float64
 4   spin1z            12741 non-null  float32
 5   H1_chisq_dof      12741 non-null  int64  
 6   L1_sg_chisq       12741 non-null  float32
 7   L1_chisq_dof      12741 non-null  int64  
 8   L1_snr            12741 non-null  float32
 9   H1_coa_phase      12741 non-null  float32
 10  L1_coa_phase      12741 non-null  float32
 11  stat              12741 non-null  float32
 12  f_lower           12741 non-null  float32
 13  far               12741 non-null  float64
 14  H1_reduced_chisq  12741 non-null  float64
 15  H1_end_time       12741 non-null  float64
 16  L1_sigmasq        12741 non-null  float3

- 对数据切片，取出部分数据特征维度

In [18]:
df_1OGC_demo = df_1OGC[['name', 'H1_snr', 'H1_end_time', 'L1_snr', 'L1_end_time', 'pastro', 'far']]
df_4OGC_demo = df_4OGC[['Name', 'GpsTime', 'SNR_H', 'SNR_L', 'Pastro', 'IFAR']]

In [19]:
df_1OGC_demo.head()

,name,H1_snr,H1_end_time,L1_snr,L1_end_time,pastro,far
0,b'150912+06:13:00UTC',4.833010,1.126074e+09,4.890781,1.126074e+09,2.420347e-11,98138.077512
1,b'150912+06:19:09UTC',5.042901,1.126074e+09,4.935375,1.126074e+09,2.322201e-10,78064.127309
2,b'150912+06:26:35UTC',5.678490,1.126074e+09,4.572548,1.126074e+09,4.259693e-10,75231.066448
3,b'150912+06:42:42UTC',4.615423,1.126075e+09,4.969444,1.126075e+09,2.420347e-11,98138.077512
4,b'150912+06:46:35UTC',5.205948,1.126076e+09,5.520049,1.126076e+09,1.452786e-05,4129.663836


In [20]:
df_4OGC_demo.head()

,Name,GpsTime,SNR_H,SNR_L,Pastro,IFAR
Number,,,,,,
1,GW150914_095045,1.126259e+09,19.934933,13.047992,1.0,16376.568631
2,GW151012_095443,1.128679e+09,6.885883,6.577003,1.0,897.275904
3,GW151226_033853,1.135136e+09,10.518603,7.405562,1.0,14799.661708
4,GW170104_101158,1.167560e+09,8.863738,9.633432,1.0,18235.991190
5,GW170121_212536,1.169069e+09,5.239263,8.938467,1.0,16.109533


## 数据清理

- 要想让两个数据产生“关联”，就一定要有唯一的“id”作为指引
    - 对 1-OGC 和 4-OGC 数据的 `name`/`Name` 特征进行数据清理
    

In [27]:
df_1OGC_demo['ID'] = df_1OGC_demo.name.map(lambda x: x.decode()[:-3].replace('+','_').replace(':', ''))
df_4OGC_demo['ID'] = df_4OGC_demo.Name.map(lambda x: x[2:])

- 分别构造一个 `datetime` 对象的数据特征，以方便解析数据

Case: 

```python
from datetime import datetime

encoded_string = b'150912+06:13:00UTC'
decoded_string = encoded_string.decode('utf-8')  # 将字节串解码为字符串
dt = datetime.strptime(decoded_string, '%y%m%d+%H:%M:%S%Z')

print(dt)
```

In [36]:
df_1OGC_demo.loc[:,'Date'] = df_1OGC_demo.name.map(lambda x: datetime.strptime(x.decode(), '%y%m%d+%H:%M:%S%Z'))
df_4OGC_demo.loc[:,'Date'] = df_4OGC_demo.Name.map(lambda x: datetime.strptime(x[2:], '%y%m%d_%H%M%S'))

- 构造一个新的 network SNR 数据特征
    - `sqrt(H1^2 + L1^2)`

In [40]:
df_1OGC_demo.loc[:,'network_snr'] = np.sqrt(df_1OGC_demo.H1_snr**2 + df_1OGC_demo.L1_snr**2)
df_4OGC_demo.loc[:,'network_snr'] = np.sqrt(df_4OGC_demo.SNR_H**2 + df_4OGC_demo.SNR_L**2)

- 整理误报率特征为统一的量纲

In [43]:
df_4OGC_demo.insert(0, 'far', 1/df_4OGC_demo.IFAR)

- 其他

In [45]:
# 去掉不要的数据特征列
df_1OGC_demo = df_1OGC_demo.drop(['name', 'H1_snr', 'L1_snr', 'H1_end_time', 'L1_end_time'], axis=1)
df_4OGC_demo = df_4OGC_demo.drop(['Name', 'IFAR', 'SNR_H', 'SNR_L', 'GpsTime'], axis=1)

In [47]:
# 将ID作为索引
df_1OGC_demo = df_1OGC_demo.set_index('ID')
df_4OGC_demo = df_4OGC_demo.set_index('ID')

In [48]:
df_1OGC_demo

,pastro,far,Date,network_snr
ID,,,,
150912_061300,2.420347e-11,98138.077512,2015-09-12 06:13:00,6.875880
150912_061909,2.322201e-10,78064.127309,2015-09-12 06:19:09,7.056116
150912_062635,4.259693e-10,75231.066448,2015-09-12 06:26:35,7.290641
150912_064242,2.420347e-11,98138.077512,2015-09-12 06:42:42,6.782146
150912_064635,1.452786e-05,4129.663836,2015-09-12 06:46:35,7.587676
...,...,...,...,...
160119_144637,2.678670e-09,71456.714968,2016-01-19 14:46:37,6.863019
160119_144735,1.076392e-09,76124.381660,2016-01-19 14:47:35,6.783543
160119_145105,7.124927e-04,277.011643,2016-01-19 14:51:05,7.661174


In [49]:
df_4OGC_demo

,far,Pastro,Date,network_snr
ID,,,,
150914_095045,0.000061,1.00,2015-09-14 09:50:45,23.825441
151012_095443,0.001114,1.00,2015-10-12 09:54:43,9.522203
151226_033853,0.000068,1.00,2015-12-26 03:38:53,12.864034
170104_101158,0.000055,1.00,2017-01-04 10:11:58,13.090793
170121_212536,0.062075,1.00,2017-01-21 21:25:36,10.360795
...,...,...,...,...
200305_084739,52.239884,0.59,2020-03-05 08:47:39,7.606598
200306_093714,56.109414,0.51,2020-03-06 09:37:14,8.028445
200311_115853,0.001224,1.00,2020-03-11 11:58:53,15.562571


## 筛选与排序

- 看看两个表里 2015 年的疑似引力波 candidates 有什么特点？

In [50]:
df_1OGC_demo.Date.sort_values()

ID
150912_061300   2015-09-12 06:13:00
150912_061909   2015-09-12 06:19:09
150912_062635   2015-09-12 06:26:35
150912_064242   2015-09-12 06:42:42
150912_064635   2015-09-12 06:46:35
                        ...        
160119_144637   2016-01-19 14:46:37
160119_144735   2016-01-19 14:47:35
160119_145105   2016-01-19 14:51:05
160119_145343   2016-01-19 14:53:43
160119_145652   2016-01-19 14:56:52
Name: Date, Length: 12741, dtype: datetime64[ns]

In [51]:
df_4OGC_demo.Date.sort_values()

ID
150914_095045   2015-09-14 09:50:45
151012_095443   2015-10-12 09:54:43
151226_033853   2015-12-26 03:38:53
170104_101158   2017-01-04 10:11:58
170121_212536   2017-01-21 21:25:36
                        ...        
200305_084739   2020-03-05 08:47:39
200306_093714   2020-03-06 09:37:14
200311_115853   2020-03-11 11:58:53
200316_215756   2020-03-16 21:57:56
200318_191337   2020-03-18 19:13:37
Name: Date, Length: 94, dtype: datetime64[ns]

In [54]:
df_4OGC_demo.Date.map(lambda x: x.year == 2015)

ID
150914_095045     True
151012_095443     True
151226_033853     True
170104_101158    False
170121_212536    False
                 ...  
200305_084739    False
200306_093714    False
200311_115853    False
200316_215756    False
200318_191337    False
Name: Date, Length: 94, dtype: bool

In [55]:
df_4OGC_2015_demo = df_4OGC_demo[df_4OGC_demo.Date.map(lambda x: x.year==2015)]
df_4OGC_2015_demo

,far,Pastro,Date,network_snr
ID,,,,
150914_095045,0.000061,1.0,2015-09-14 09:50:45,23.825441
151012_095443,0.001114,1.0,2015-10-12 09:54:43,9.522203
151226_033853,0.000068,1.0,2015-12-26 03:38:53,12.864034


In [58]:
condition = df_1OGC_demo.Date.map(lambda x: (x.year==2015))
condition &= df_1OGC_demo.pastro > 1e-2
df_1OGC_2015_demo = df_1OGC_demo[condition]
df_1OGC_2015_demo

,pastro,far,Date,network_snr
ID,,,,
150914_095045,1.000000,0.000015,2015-09-14 09:50:45,23.790823
151012_095443,0.975890,0.002240,2015-10-12 09:54:43,9.662005
151213_001220,0.046571,3.239747,2015-12-13 00:12:20,9.211678
151216_184930,0.017194,9.418957,2015-12-16 18:49:30,8.560873
151222_052826,0.012046,13.309665,2015-12-22 05:28:26,8.600986
151226_033853,1.000000,0.000017,2015-12-26 03:38:53,13.054273


## 数据合并

- 尝试将两个数据有着相同的 特征列 的数据直接合并一下

In [59]:
df_1OGC_2015_demo[['Date']]

,Date
ID,
150914_095045,2015-09-14 09:50:45
151012_095443,2015-10-12 09:54:43
151213_001220,2015-12-13 00:12:20
151216_184930,2015-12-16 18:49:30
151222_052826,2015-12-22 05:28:26
151226_033853,2015-12-26 03:38:53


In [60]:
df_4OGC_2015_demo[['Date']]

,Date
ID,
150914_095045,2015-09-14 09:50:45
151012_095443,2015-10-12 09:54:43
151226_033853,2015-12-26 03:38:53


In [61]:
pd.concat([df_1OGC_2015_demo[['Date']], df_4OGC_2015_demo[['Date']]])

,Date
ID,
150914_095045,2015-09-14 09:50:45
151012_095443,2015-10-12 09:54:43
151213_001220,2015-12-13 00:12:20
151216_184930,2015-12-16 18:49:30
151222_052826,2015-12-22 05:28:26
151226_033853,2015-12-26 03:38:53
150914_095045,2015-09-14 09:50:45
151012_095443,2015-10-12 09:54:43
151226_033853,2015-12-26 03:38:53


这时你会发现index索引项不唯一了。按照某个日期取出的数据不唯一，且无法区分是哪个是来自哪个catalog

In [62]:
_.loc['150914_095045']

,Date
ID,
150914_095045,2015-09-14 09:50:45
150914_095045,2015-09-14 09:50:45


这时，如果想区分该怎么办呢？可以通过指定“keys”参数来实现（无脑的 concat 开始变聪明）

In [63]:
pd.concat([
    df_1OGC_2015_demo[['Date']],
    df_4OGC_2015_demo[['Date']]
], keys=['1-OGC', '4-OGC'])

Date
      ID                               
1-OGC 150914_095045 2015-09-14 09:50:45
      151012_095443 2015-10-12 09:54:43
      151213_001220 2015-12-13 00:12:20
      151216_184930 2015-12-16 18:49:30
      151222_052826 2015-12-22 05:28:26
      151226_033853 2015-12-26 03:38:53
4-OGC 150914_095045 2015-09-14 09:50:45
      151012_095443 2015-10-12 09:54:43
      151226_033853 2015-12-26 03:38:53

同理，纵向也可以合并。通过指定“axis”参数来实现。

In [64]:
pd.concat([df_1OGC_2015_demo[['Date']], df_4OGC_2015_demo[['Date']]], axis=1)

,Date,Date
ID,,
150914_095045,2015-09-14 09:50:45,2015-09-14 09:50:45
151012_095443,2015-10-12 09:54:43,2015-10-12 09:54:43
151213_001220,2015-12-13 00:12:20,NaT
151216_184930,2015-12-16 18:49:30,NaT
151222_052826,2015-12-22 05:28:26,NaT
151226_033853,2015-12-26 03:38:53,2015-12-26 03:38:53


也同理，纵向也可以通过指定“keys”参数，来区分出level

In [65]:
pd.concat([df_1OGC_2015_demo[['Date']], df_4OGC_2015_demo[['Date']]], axis=1, keys=['1-OGC', '4-OGC'])

,1-OGC,4-OGC
,Date,Date
ID,,
150914_095045,2015-09-14 09:50:45,2015-09-14 09:50:45
151012_095443,2015-10-12 09:54:43,2015-10-12 09:54:43
151213_001220,2015-12-13 00:12:20,NaT
151216_184930,2015-12-16 18:49:30,NaT
151222_052826,2015-12-22 05:28:26,NaT
151226_033853,2015-12-26 03:38:53,2015-12-26 03:38:53


下面我们再来熟悉一下使用merge的合并操作

In [66]:
# 重置索引
df_1OGC_2015_re = df_1OGC_2015_demo[['Date', 'pastro']].reset_index()
df_4OGC_2015_re = df_4OGC_2015_demo[['Date', 'Pastro']].reset_index()
df_1OGC_2015_re.shape, df_4OGC_2015_re.shape

((6, 3), (3, 3))

In [67]:
df_1OGC_2015_re

,ID,Date,pastro
0,150914_095045,2015-09-14 09:50:45,1.000000
1,151012_095443,2015-10-12 09:54:43,0.975890
2,151213_001220,2015-12-13 00:12:20,0.046571
3,151216_184930,2015-12-16 18:49:30,0.017194
4,151222_052826,2015-12-22 05:28:26,0.012046
5,151226_033853,2015-12-26 03:38:53,1.000000


In [68]:
df_4OGC_2015_re

,ID,Date,Pastro
0,150914_095045,2015-09-14 09:50:45,1.0
1,151012_095443,2015-10-12 09:54:43,1.0
2,151226_033853,2015-12-26 03:38:53,1.0


直接执行merge合并操作，并思考一下合并结果为什么只有3行数据？

In [69]:
pd.merge(df_1OGC_2015_re, df_4OGC_2015_re)

,ID,Date,pastro,Pastro
0,150914_095045,2015-09-14 09:50:45,1.00000,1.0
1,151012_095443,2015-10-12 09:54:43,0.97589,1.0
2,151226_033853,2015-12-26 03:38:53,1.00000,1.0


merge函数可以指定更多参数来实现各种不同的合并方式。how是inner内连接，on是指定用于连接的字段名。

In [70]:
pd.merge(df_1OGC_2015_re, df_4OGC_2015_re, on='ID', how='inner')

,ID,Date_x,pastro,Date_y,Pastro
0,150914_095045,2015-09-14 09:50:45,1.00000,2015-09-14 09:50:45,1.0
1,151012_095443,2015-10-12 09:54:43,0.97589,2015-10-12 09:54:43,1.0
2,151226_033853,2015-12-26 03:38:53,1.00000,2015-12-26 03:38:53,1.0


In [71]:
pd.merge(df_1OGC_2015_re, df_4OGC_2015_re, on=['ID', 'Date'], how='inner')

,ID,Date,pastro,Pastro
0,150914_095045,2015-09-14 09:50:45,1.00000,1.0
1,151012_095443,2015-10-12 09:54:43,0.97589,1.0
2,151226_033853,2015-12-26 03:38:53,1.00000,1.0


再看一下how=outer外连接的合并结果，可以对比观察一下内外连接的不同。

In [72]:
pd.merge(df_1OGC_2015_re, df_4OGC_2015_re, on=['ID', 'Date'], how='outer')

,ID,Date,pastro,Pastro
0,150914_095045,2015-09-14 09:50:45,1.000000,1.0
1,151012_095443,2015-10-12 09:54:43,0.975890,1.0
2,151213_001220,2015-12-13 00:12:20,0.046571,NaN
3,151216_184930,2015-12-16 18:49:30,0.017194,NaN
4,151222_052826,2015-12-22 05:28:26,0.012046,NaN
5,151226_033853,2015-12-26 03:38:53,1.000000,1.0


## 透视表

* 也许大多数人都有在Excel中使用数据透视表的经历，其实Pandas也提供了一个类似的功能，名为 `pivot_table`。
* 将一个DataFrame的记录数据整合成表格(类似Excel中的数据透视表功能)，而且是按照pivot(‘index=xx’,’columns=xx’,’values=xx’)来整合的。

为了进行透视表的操作，我们先做些数据准备：
* 插入一个名为“Obs”的字段分别标识不同catalog来源的数据；
* 合并两个catalog的 2015 年的数据（pastro>0.01），并按索引值排序；
* 重置合并数据的索引；

In [73]:
df_1OGC_2015_demo.insert(0, 'Obs', '1-OGC')
df_4OGC_2015_demo.insert(0, 'Obs', '4-OGC')

combined = pd.concat([df_1OGC_2015_demo[['Obs', 'Date', 'network_snr', 'far']], 
                      df_4OGC_2015_demo[['Obs', 'Date', 'network_snr', 'far']]]).sort_index()

combined = combined.reset_index();

In [74]:
combined

,ID,Obs,Date,network_snr,far
0,150914_095045,1-OGC,2015-09-14 09:50:45,23.790823,0.000015
1,150914_095045,4-OGC,2015-09-14 09:50:45,23.825441,0.000061
2,151012_095443,1-OGC,2015-10-12 09:54:43,9.662005,0.002240
3,151012_095443,4-OGC,2015-10-12 09:54:43,9.522203,0.001114
4,151213_001220,1-OGC,2015-12-13 00:12:20,9.211678,3.239747
5,151216_184930,1-OGC,2015-12-16 18:49:30,8.560873,9.418957
6,151222_052826,1-OGC,2015-12-22 05:28:26,8.600986,13.309665
7,151226_033853,1-OGC,2015-12-26 03:38:53,13.054273,0.000017
8,151226_033853,4-OGC,2015-12-26 03:38:53,12.864034,0.000068


基于上面的数据，制作生成一张简单的数据透视表

In [75]:
network_snr = combined.pivot(index='ID', columns='Obs', values='network_snr')
network_snr

Obs,1-OGC,4-OGC
ID,,
150914_095045,23.790823,23.825441
151012_095443,9.662005,9.522203
151213_001220,9.211678,NaN
151216_184930,8.560873,NaN
151222_052826,8.600986,NaN
151226_033853,13.054273,12.864034


### **堆叠**

DataFrame 在操作上和 pivot 功能类似的函数还有： .stack() 和 .unstack(). 

这个函数可以进一步将 column 转为新一层级的 index。


In [77]:
stacked_network_snr = network_snr.stack()

stacked_network_snr

ID             Obs  
150914_095045  1-OGC    23.790823
               4-OGC    23.825441
151012_095443  1-OGC     9.662005
               4-OGC     9.522203
151213_001220  1-OGC     9.211678
151216_184930  1-OGC     8.560873
151222_052826  1-OGC     8.600986
151226_033853  1-OGC    13.054273
               4-OGC    12.864034
dtype: float64

In [78]:
type(stacked_network_snr)

pandas.core.series.Series

In [79]:
stacked_network_snr.index

MultiIndex([('150914_095045', '1-OGC'),
            ('150914_095045', '4-OGC'),
            ('151012_095443', '1-OGC'),
            ('151012_095443', '4-OGC'),
            ('151213_001220', '1-OGC'),
            ('151216_184930', '1-OGC'),
            ('151222_052826', '1-OGC'),
            ('151226_033853', '1-OGC'),
            ('151226_033853', '4-OGC')],
           names=['ID', 'Obs'])

再来看看 stacked 的数据表如何切片？

In [82]:
stacked_network_snr['150914_095045']['1-OGC']
stacked_network_snr['150914_095045']['4-OGC']

23.825440832827386

stacked 的数据又该如何 unstacked 回来？

In [83]:
unstacked_network_snr = stacked_network_snr.unstack()

unstacked_network_snr

Obs,1-OGC,4-OGC
ID,,
150914_095045,23.790823,23.825441
151012_095443,9.662005,9.522203
151213_001220,9.211678,NaN
151216_184930,8.560873,NaN
151222_052826,8.600986,NaN
151226_033853,13.054273,12.864034


### 重塑
* melt() 可以把一个 dataframe 转换成每一行都有一个唯一id标识的变量组合的数据格式
* 数据分析的时候经常要把宽数据--->>长数据（列转行）,change the DataFrame format from wide to long，有点像我们用excel 做逆透视的过程

In [84]:
combined # 这里不存在唯一id标识的特征列

,ID,Obs,Date,network_snr,far
0,150914_095045,1-OGC,2015-09-14 09:50:45,23.790823,0.000015
1,150914_095045,4-OGC,2015-09-14 09:50:45,23.825441,0.000061
2,151012_095443,1-OGC,2015-10-12 09:54:43,9.662005,0.002240
3,151012_095443,4-OGC,2015-10-12 09:54:43,9.522203,0.001114
4,151213_001220,1-OGC,2015-12-13 00:12:20,9.211678,3.239747
5,151216_184930,1-OGC,2015-12-16 18:49:30,8.560873,9.418957
6,151222_052826,1-OGC,2015-12-22 05:28:26,8.600986,13.309665
7,151226_033853,1-OGC,2015-12-26 03:38:53,13.054273,0.000017
8,151226_033853,4-OGC,2015-12-26 03:38:53,12.864034,0.000068


In [85]:
pd.melt(combined, id_vars=['Date', 'Obs'])

,Date,Obs,variable,value
0,2015-09-14 09:50:45,1-OGC,ID,150914_095045
1,2015-09-14 09:50:45,4-OGC,ID,150914_095045
2,2015-10-12 09:54:43,1-OGC,ID,151012_095443
3,2015-10-12 09:54:43,4-OGC,ID,151012_095443
4,2015-12-13 00:12:20,1-OGC,ID,151213_001220
5,2015-12-16 18:49:30,1-OGC,ID,151216_184930
6,2015-12-22 05:28:26,1-OGC,ID,151222_052826
7,2015-12-26 03:38:53,1-OGC,ID,151226_033853
8,2015-12-26 03:38:53,4-OGC,ID,151226_033853
9,2015-09-14 09:50:45,1-OGC,network_snr,23.790823


![](https://vip2.loli.io/2023/12/02/o5IUXDwP2nVG8Ym.jpg)

- 一维的逆透视（明细记录表）是用来详细记录信息的，是数据库
- 我们在做数据管理的时候，基础数据的存储一定要采用一维表。

### 分组

* 在日常的数据分析中，经常需要将数据根据某个（多个）字段划分为不同的群体（group）进行分析，如电商领域将全国的总销售额根据省份进行划分，分析各省销售额的变化情况，社交领域将用户根据画像（性别、年龄）进行细分，研究用户的使用情况和偏好等;
* 在Pandas中，上述的数据处理操作主要运用groupby完成;
* 在对于一个dataframe调用了groupby()方法的结果不是实际的分组数据,而是会得到一个DataFrameGroupBy对象;


先准备分组练习的数据，已经重置索引

In [86]:
tmp = combined.copy()
tmp.drop('ID', axis=1, inplace=True)
tmp

,Obs,Date,network_snr,far
0,1-OGC,2015-09-14 09:50:45,23.790823,0.000015
1,4-OGC,2015-09-14 09:50:45,23.825441,0.000061
2,1-OGC,2015-10-12 09:54:43,9.662005,0.002240
3,4-OGC,2015-10-12 09:54:43,9.522203,0.001114
4,1-OGC,2015-12-13 00:12:20,9.211678,3.239747
5,1-OGC,2015-12-16 18:49:30,8.560873,9.418957
6,1-OGC,2015-12-22 05:28:26,8.600986,13.309665
7,1-OGC,2015-12-26 03:38:53,13.054273,0.000017
8,4-OGC,2015-12-26 03:38:53,12.864034,0.000068


combined插入两列数据，分别是“年”和“月”：

In [87]:
tmp.insert(1, 'Year', pd.DatetimeIndex(tmp['Date']).year)
tmp.insert(2, 'Month',pd.DatetimeIndex(tmp['Date']).month)
tmp.drop('Date', axis=1, inplace=True)
tmp

,Obs,Year,Month,network_snr,far
0,1-OGC,2015,9,23.790823,0.000015
1,4-OGC,2015,9,23.825441,0.000061
2,1-OGC,2015,10,9.662005,0.002240
3,4-OGC,2015,10,9.522203,0.001114
4,1-OGC,2015,12,9.211678,3.239747
5,1-OGC,2015,12,8.560873,9.418957
6,1-OGC,2015,12,8.600986,13.309665
7,1-OGC,2015,12,13.054273,0.000017
8,4-OGC,2015,12,12.864034,0.000068


- 直接试一下 groupby 感受一下

In [88]:
grouped = tmp.groupby('Obs')
grouped

In [89]:
type(grouped.groups)

pandas.io.formats.printing.PrettyDict

In [90]:
grouped.size()  # a nice summary of the size of all the groups

Obs
1-OGC    6
4-OGC    3
dtype: int64

In [91]:
grouped.groups  # Dict {group name -> group labels}.

{'1-OGC': [0, 2, 4, 5, 6, 7], '4-OGC': [1, 3, 8]}

In [92]:
for name, group in grouped:
    print(name)
    print(group.head())

1-OGC
     Obs  Year  Month  network_snr        far
0  1-OGC  2015      9    23.790823   0.000015
2  1-OGC  2015     10     9.662005   0.002240
4  1-OGC  2015     12     9.211678   3.239747
5  1-OGC  2015     12     8.560873   9.418957
6  1-OGC  2015     12     8.600986  13.309665
4-OGC
     Obs  Year  Month  network_snr       far
1  4-OGC  2015      9    23.825441  0.000061
3  4-OGC  2015     10     9.522203  0.001114
8  4-OGC  2015     12    12.864034  0.000068


总结来说，groupby的过程就是将原有的DataFrame按照groupby的字段（这里是“Obs”），划分为若干个分组DataFrame，被分为多少个组就有多少个分组DataFrame。

所以说，在groupby之后的一系列操作（如agg、apply等），均是基于子DataFrame的操作。理解了这点，也就基本摸清了Pandas中groupby操作的主要原理。

### 聚合

理解了分组之后，我们开始了解聚合 aggregate 。

聚合操作是groupby后非常常见的操作，会写SQL的朋友对此应该是非常熟悉了。聚合操作可以用来求和、均值、最大值、最小值等。

In [94]:
tmp.groupby(['Obs']).agg('max')

,Year,Month,network_snr,far
Obs,,,,
1-OGC,2015,12,23.790823,13.309665
4-OGC,2015,12,23.825441,0.001114


In [95]:
# 先根据 Obs 的取值分组，每组构建 ['Year', 'network_snr'] 为列的DataFrame，每列取出 max
tmp.groupby(['Obs'])[['Year', 'network_snr']].agg('max')

,Year,network_snr
Obs,,
1-OGC,2015,23.790823
4-OGC,2015,23.825441


In [96]:
# 按顺序分别对 Obs，Year，Month 组层细分的分组，每组构建 DataFrame，每列取出max
tmp.groupby(['Obs', 'Year', 'Month']).agg('max')

network_snr        far
Obs   Year Month                        
1-OGC 2015 9        23.790823   0.000015
           10        9.662005   0.002240
           12       13.054273  13.309665
4-OGC 2015 9        23.825441   0.000061
           10        9.522203   0.001114
           12       12.864034   0.000068

In [97]:
# 按顺序分别对 Obs，Year，Month 组层细分的分组，每组构建 DataFrame，每列取出max和min
tmp.groupby(['Obs', 'Year', 'Month']).agg(['max', 'min'])

network_snr                   far          
                         max        min        max       min
Obs   Year Month                                            
1-OGC 2015 9       23.790823  23.790823   0.000015  0.000015
           10       9.662005   9.662005   0.002240  0.002240
           12      13.054273   8.560873  13.309665  0.000017
4-OGC 2015 9       23.825441  23.825441   0.000061  0.000061
           10       9.522203   9.522203   0.001114  0.001114
           12      12.864034  12.864034   0.000068  0.000068

In [98]:
# 按顺序分别对 Obs，Year，Month 组层细分的分组，每组构建 DataFrame，每列取出max和min，最后把 Month 这一行转换为列。
tmp.groupby(['Obs', 'Year', 'Month']).agg(['max', 'min']).unstack('Month')

network_snr                                                       \
                   max                             min                        
Month               9         10         12         9         10         12   
Obs   Year                                                                    
1-OGC 2015   23.790823  9.662005  13.054273  23.790823  9.662005   8.560873   
4-OGC 2015   23.825441  9.522203  12.864034  23.825441  9.522203  12.864034   

                 far                                                     
                 max                            min                      
Month             9         10         12        9         10        12  
Obs   Year                                                               
1-OGC 2015  0.000015  0.002240  13.309665  0.000015  0.002240  0.000017  
4-OGC 2015  0.000061  0.001114   0.000068  0.000061  0.001114  0.000068

综合本节所学的知识，思考一下：上面的透视表可以直接一个函数搞定么？

> 可以！

In [99]:
pd.pivot_table(tmp, index=['Obs', 'Year'], values=['network_snr','far'], columns=['Month'], aggfunc=['max', 'min'])

max                                                        \
                 far                      network_snr                        
Month             9         10         12          9         10         12   
Obs   Year                                                                   
1-OGC 2015  0.000015  0.002240  13.309665   23.790823  9.662005  13.054273   
4-OGC 2015  0.000061  0.001114   0.000068   23.825441  9.522203  12.864034   

                 min                                                       
                 far                     network_snr                       
Month             9         10        12          9         10         12  
Obs   Year                                                                 
1-OGC 2015  0.000015  0.002240  0.000017   23.790823  9.662005   8.560873  
4-OGC 2015  0.000061  0.001114  0.000068   23.825441  9.522203  12.864034